In [34]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from math import gcd
import pandas as pd
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit import *
from qiskit.tools import job_monitor
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import qiskit.tools.jupyter
from math import *
from random import randint

In [35]:
def initialize_qubits(given_circuit, n, m):

    given_circuit.h(range(n))
    given_circuit.x(n+m-1)
#
def c_amod15(a, x):
    if a not in [2,7,8,11,13]:
        raise ValueError("'a' must be 2,7,8,11,13")
    U = QuantumCircuit(4)        
    for iteration in range(x):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a == 11:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U.draw()
    return U.control()
#
def modular_exponentiation(circuit, n, m, a):
    

    for x in range(n):
        exponent = 2**x
        circuit.append(c_amod15(a, exponent), [x] + list(range(n, n+m)))
#
def inverse_qft(circuit, measurement_qubits):
    

    circuit.append(QFT( len(measurement_qubits), do_swaps=False).inverse(), measurement_qubits)
#
def shors_algorithm(n, m, a):
    
   
    qc = QuantumCircuit(n+m, n)
    
  
    initialize_qubits(qc, n, m)
    qc.barrier()

   
    modular_exponentiation(qc, n, m, a)
    qc.barrier()

 
    inverse_qft(qc, range(n))

    
    qc.measure(range(n), range(n))
    
    return qc

In [41]:
n = 3; m = 4; a = 11
final_circuit = shors_algorithm(n, m, a)
trans_circuit = transpile(final_circuit, basis_gates = ['u', 'cx'])
trans_circuit.draw(fold = 100)

┌────────────┐ ░                                                                »
q_0: ┤ U(π/2,0,π) ├─░──────────────────────────────────────────■─────────────────────»
     ├────────────┤ ░                                          │                     »
q_1: ┤ U(π/2,0,π) ├─░──────────────────────────────────────────┼─────────────────────»
     ├────────────┤ ░                                          │                     »
q_2: ┤ U(π/2,0,π) ├─░──────────────────────────────────────────┼─────────────────────»
     └────────────┘ ░                                          │                     »
q_3: ───────────────░─────────────────■────────────────────────┼──────────────────■──»
                    ░                 │                        │                  │  »
q_4: ───────────────░─────────────────┼────■───────────────────┼──────────────────┼──»
                    ░ ┌────────────┐┌─┴─┐  │  ┌─────────────┐┌─┴─┐┌────────────┐┌─┴─┐»
q_5: ───────────────░─┤ U(π/2,0,π) ├┤ X ├──┼──┤ U(0,0,-π/4) ├┤ X ├┤ U(0,0,π/4) ├┤ X ├»
      ┌──────────┐  ░ ├────────────┤└───┘┌─┴─┐├─────────────┤└───┘└────────────┘└───┘»
q_6: ─┤ U(π,0,π) ├──░─┤ U(π/2,0,π) ├─────┤ X ├┤ U(0,0,-π/4) ├────────────────────────»
      └──────────┘  ░ └────────────┘     └───┘└─────────────┘                        »
c: 3/════════════════════════════════════════════════════════════════════════════════»
                                                                                     »
«                                            ┌────────────┐                                       »
«q_0: ─────────────────■──────────■──────────┤ U(0,0,π/4) ├──■────────────────────────────────────»
«                      │          │          └────────────┘  │                                    »
«q_1: ─────────────────┼──────────┼──────────────────────────┼────────────────────────────────────»
«                      │          │                          │                                    »
«q_2: ─────────────────┼──────────┼──────────────────────────┼────────────────────────────────────»
«      ┌────────────┐  │        ┌─┴─┐       ┌─────────────┐┌─┴─┐┌────────────┐┌───┐┌─────────────┐»
«q_3: ─┤ U(0,0,π/4) ├──┼────────┤ X ├───────┤ U(0,0,-π/4) ├┤ X ├┤ U(π/2,0,π) ├┤ X ├┤ U(0,0,-π/4) ├»
«      └────────────┘  │        └───┘       └─────────────┘└───┘└────────────┘└─┬─┘└─────────────┘»
«q_4: ─────────────────┼────────────────────────────────────────────────────────┼─────────────────»
«     ┌─────────────┐┌─┴─┐┌────────────────┐                                    │                 »
«q_5: ┤ U(0,0,-π/4) ├┤ X ├┤ U(π/2,0,-3π/4) ├────────────────────────────────────■─────────────────»
«     └─────────────┘└───┘└────────────────┘                                                      »
«q_6: ────────────────────────────────────────────────────────────────────────────────────────────»
«                                                                                                 »
«c: 3/════════════════════════════════════════════════════════════════════════════════════════════»
«                                                                                                 »
«                                                                         ┌────────────┐     »
«q_0: ──■──────────────────────────────────────■──────────────────────■───┤ U(0,0,π/4) ├──■──»
«       │                                      │                      │   └────────────┘  │  »
«q_1: ──┼──────────────────────────────────────┼──────────────────────┼───────────────────┼──»
«       │                                      │                      │                   │  »
«q_2: ──┼──────────────────────────────────────┼──────────────────────┼───────────────────┼──»
«     ┌─┴─┐┌────────────┐┌───┐┌─────────────┐┌─┴─┐┌────────────────┐  │                   │  »
«q_3: ┤ X ├┤ U(0,0,π/4) ├┤ X ├┤ U(0,0,-π/4) ├┤ X ├┤ U(π/2,0,-3π/4) ├──┼───────────────────┼──»
«     └───┘└────────────┘└─┬─┘└─────────────┘└───┘└───────────────

In [37]:
simulator = Aer.get_backend('qasm_simulator')
counts = execute(final_circuit, backend=simulator, shots=1000).result().get_counts(final_circuit)

In [22]:
for i in counts:
    measured_value = int(i[::-1], 2)
  
    
    if measured_value % 2 != 0:
        print("Measured value not even")
        continue #measured value should be even as we are doing a^(r/2) mod N and r/2 should be int
    x = int((a ** (measured_value/2)) % 15)
    if (x + 1) % 15 == 0:
        continue
    factors = gcd(x + 1, 15), gcd(x - 1, 15) #we saw earlier that a^(r/2)+1 or a^(r/2)-1 should be a factor of 15
    print(factors)

Measured value not even
Measured value not even
(1, 15)
(1, 15)
(3, 5)
(3, 5)
Measured value not even
